# Imports and utility functions

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import matplotlib.pyplot as plt # plotting
import matplotlib.image as mpimg # images
import numpy as np #numpy
import seaborn as sns
import tensorflow.compat.v2 as tf #use tensorflow v2 as a main 
import tensorflow.keras as keras # required for high level applications
from sklearn.model_selection import train_test_split # split for validation sets
from sklearn.metrics import accuracy_score, f1_score, classification_report
import scipy
import pandas as pd
import re

In [ ]:
def cleanTexts(texts):
    cleaned = []
    pattern = "[^a-zA-Z0-9]"
    for text in texts:
        clrd = re.sub(pattern," ",text).lower().strip()
        cleaned.append(clrd)
    return cleaned

# Load data

In [ ]:
from sklearn.utils import shuffle

dataset = pd.read_csv('train_data_tweetsENG.csv')
dataset = shuffle(dataset)
dataset.tail()

,text,label
699630,"at kitty house except that xani is here too, ...",1
243920,@jordanknight U 5 guys could be standing in th...,1
211905,I really do love it here. I had a blast at the...,0
406562,@christamacphee I'm a future teacher -- I'm fi...,1
277677,oh can you believe it's sunday night again!! u...,0


In [ ]:
x_train = list(cleanTexts(dataset['text']))
# print(x[:5])

y_train = list(dataset['label'])
# print(y[:5])

#x_train = x_train[:20000]
#y_train = y_train[:20000]

#x_train = x_train[:500000]
#y_train = y_train[:500000]

x_train = x_train[:900000]
y_train = y_train[:900000]

In [ ]:
test_data = pd.read_csv('test_data_tweetsENG.csv')

x_test = list(cleanTexts(test_data['text']))
y_test = list(test_data['label'])

x_test = x_test[:10000]
y_test = y_test[:10000]

# Train model and predict on test dataset

In [ ]:
from tensorflow import string as tf_string
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.compat.v1.keras.layers import CuDNNGRU, CuDNNLSTM
from tensorflow.keras.layers import LSTM, GRU, Bidirectional

# 1 - 6 Vectorizer parameters

In [ ]:
embedding_dim = 128 # Dimension of embedded representation 
vocab_size = 30000 # Number of unique tokens in vocabulary
sequence_length = 30 # Output dimension after vectorizing 

vect_layer = TextVectorization(max_tokens=vocab_size, output_mode='int', output_sequence_length=sequence_length)
vect_layer.adapt(x_train)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.10, random_state=69, stratify = y_train)

## 1.Experiment

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_v = vect_layer(input_layer)
emb = keras.layers.Embedding(vocab_size, embedding_dim)(x_v)
x = LSTM(64, activation='relu', return_sequences=True)(emb)
x = GRU(64, activation='relu', return_sequences=True)(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(64, 'relu')(x)
x = keras.layers.Dense(32, 'relu')(x)
x = keras.layers.Dropout(0.2)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(optimizer=optimizer, loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 30)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 30, 128)           3840000   
                                                                 
 lstm (LSTM)                 (None, 30, 64)            49408     
                                                                 
 gru (GRU)                   (None, 30, 64)            24960     
                                                                 
 flatten (Flatten)           (None, 1920)              0         
                                                             

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

batch_size = 768
epochs = 5
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), callbacks=[es], epochs=epochs, batch_size=batch_size)

Epoch 1/5
30/30 [==============================] - 12s 152ms/step - loss: 0.6710 - accuracy: 0.5772 - val_loss: 0.5785 - val_accuracy: 0.6992
Epoch 2/5
30/30 [==============================] - 4s 134ms/step - loss: 0.4860 - accuracy: 0.7731 - val_loss: 0.5283 - val_accuracy: 0.7436
Epoch 3/5
30/30 [==============================] - 4s 132ms/step - loss: 0.3351 - accuracy: 0.8652 - val_loss: 0.5604 - val_accuracy: 0.7336
Epoch 4/5
30/30 [==============================] - 4s 135ms/step - loss: 0.2320 - accuracy: 0.9157 - val_loss: 0.6540 - val_accuracy: 0.7376
Epoch 5/5
30/30 [==============================] - 4s 136ms/step - loss: 0.1634 - accuracy: 0.9415 - val_loss: 0.7466 - val_accuracy: 0.7276


In [ ]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 5s 15ms/step - loss: 0.7370 - accuracy: 0.7257


[0.7370451092720032, 0.7257000207901001]

In [ ]:
y_pred=model.predict(x_test)

accuracy_sc = accuracy_score(y_pred=y_pred.round(),y_true=y_test)*100
f1_sc = f1_score(y_pred=y_pred.round(),y_true=y_test)

print("Accuracy score is {}% ".format(accuracy_sc))
print("f1-score is {}% ".format(f1_sc))
print(classification_report(y_pred=y_pred.round(),y_true=y_test))

Accuracy score is 72.57000000000001% 
f1-score is 0.7327292214752023% 
              precision    recall  f1-score   support

           0       0.74      0.70      0.72      5000
           1       0.71      0.75      0.73      5000

    accuracy                           0.73     10000
   macro avg       0.73      0.73      0.73     10000
weighted avg       0.73      0.73      0.73     10000



## 2.Experiment

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_v = vect_layer(input_layer)
emb = keras.layers.Embedding(vocab_size, embedding_dim)(x_v)
x = Bidirectional(LSTM(64, activation='relu', return_sequences=True))(emb)
x = Bidirectional(GRU(64, activation='relu', return_sequences=True))(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(64, 'relu')(x)
x = keras.layers.Dense(32, 'relu')(x)
x = keras.layers.Dropout(0.2)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(optimizer=optimizer, loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 30)               0         
 torization)                                                     
                                                                 
 embedding_3 (Embedding)     (None, 30, 128)           3840000   
                                                                 
 bidirectional_4 (Bidirectio  (None, 30, 128)          98816     
 nal)                                                            
                                                                 
 bidirectional_5 (Bidirectio  (None, 30, 128)          74496     
 nal)                                                            
                                                           

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

batch_size = 128
epochs = 2
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), callbacks=[es], epochs=epochs, batch_size=batch_size)

Epoch 1/2
176/176 [==============================] - 47s 244ms/step - loss: 0.5743 - accuracy: 0.6852 - val_loss: 0.4988 - val_accuracy: 0.7640
Epoch 2/2
176/176 [==============================] - 41s 232ms/step - loss: 0.3777 - accuracy: 0.8400 - val_loss: 0.5400 - val_accuracy: 0.7540


In [ ]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 6s 20ms/step - loss: 0.5293 - accuracy: 0.7515


[0.5292622447013855, 0.7515000104904175]

In [ ]:
y_pred=model.predict(x_test)

accuracy_sc = accuracy_score(y_pred=y_pred.round(),y_true=y_test)*100
f1_sc = f1_score(y_pred=y_pred.round(),y_true=y_test)

print("Accuracy score is {}% ".format(accuracy_sc))
print("f1-score is {}% ".format(f1_sc))
print(classification_report(y_pred=y_pred.round(),y_true=y_test))

Accuracy score is 75.14999999999999% 
f1-score is 0.7552929591334319% 
              precision    recall  f1-score   support

           0       0.76      0.74      0.75      5000
           1       0.74      0.77      0.76      5000

    accuracy                           0.75     10000
   macro avg       0.75      0.75      0.75     10000
weighted avg       0.75      0.75      0.75     10000



## 3.Experiment

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_v = vect_layer(input_layer)
emb = keras.layers.Embedding(vocab_size, embedding_dim)(x_v)
x = LSTM(64, activation='relu', return_sequences=True)(emb)
x = GRU(64, activation='relu', return_sequences=True)(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(64, 'relu')(x)
x = keras.layers.Dense(32, 'relu')(x)
x = keras.layers.Dropout(0.2)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(optimizer=optimizer, loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 30)               0         
 ectorization)                                                   
                                                                 
 embedding_5 (Embedding)     (None, 30, 128)           3840000   
                                                                 
 lstm_5 (LSTM)               (None, 30, 64)            49408     
                                                                 
 gru_5 (GRU)                 (None, 30, 64)            24960     
                                                                 
 flatten_5 (Flatten)         (None, 1920)              0         
                                                           

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

batch_size = 768
epochs = 2
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), callbacks=[es], epochs=epochs, batch_size=batch_size)

Epoch 1/2
586/586 [==============================] - 84s 139ms/step - loss: 0.4588 - accuracy: 0.7814 - val_loss: 0.4109 - val_accuracy: 0.8114
Epoch 2/2
586/586 [==============================] - 84s 143ms/step - loss: 0.3863 - accuracy: 0.8267 - val_loss: 0.4036 - val_accuracy: 0.8149


In [ ]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 4s 14ms/step - loss: 0.4060 - accuracy: 0.8093


[0.4059540331363678, 0.8093000054359436]

In [ ]:
y_pred=model.predict(x_test)

accuracy_sc = accuracy_score(y_pred=y_pred.round(),y_true=y_test)*100
f1_sc = f1_score(y_pred=y_pred.round(),y_true=y_test)

print("Accuracy score is {}% ".format(accuracy_sc))
print("f1-score is {}% ".format(f1_sc))
print(classification_report(y_pred=y_pred.round(),y_true=y_test))

Accuracy score is 80.93% 
f1-score is 0.814620394672888% 
              precision    recall  f1-score   support

           0       0.83      0.78      0.80      5000
           1       0.79      0.84      0.81      5000

    accuracy                           0.81     10000
   macro avg       0.81      0.81      0.81     10000
weighted avg       0.81      0.81      0.81     10000



## 4.Experiment

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_v = vect_layer(input_layer)
emb = keras.layers.Embedding(vocab_size, embedding_dim)(x_v)
x = Bidirectional(LSTM(64, activation='relu', return_sequences=True))(emb)
x = Bidirectional(GRU(64, activation='relu', return_sequences=True))(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(64, 'relu')(x)
x = keras.layers.Dense(32, 'relu')(x)
x = keras.layers.Dropout(0.2)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(optimizer=optimizer, loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 30)               0         
 ectorization)                                                   
                                                                 
 embedding_7 (Embedding)     (None, 30, 128)           3840000   
                                                                 
 bidirectional_8 (Bidirectio  (None, 30, 128)          98816     
 nal)                                                            
                                                                 
 bidirectional_9 (Bidirectio  (None, 30, 128)          74496     
 nal)                                                            
                                                           

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

batch_size = 768
epochs = 2
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), callbacks=[es], epochs=epochs, batch_size=batch_size)

Epoch 1/2
586/586 [==============================] - 151s 252ms/step - loss: 0.4534 - accuracy: 0.7860 - val_loss: 0.4098 - val_accuracy: 0.8112
Epoch 2/2
586/586 [==============================] - 144s 246ms/step - loss: 0.3830 - accuracy: 0.8280 - val_loss: 0.3997 - val_accuracy: 0.8166


In [ ]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 6s 19ms/step - loss: 0.4067 - accuracy: 0.8112


[0.4067350924015045, 0.8112000226974487]

In [ ]:
y_pred=model.predict(x_test)

accuracy_sc = accuracy_score(y_pred=y_pred.round(),y_true=y_test)*100
f1_sc = f1_score(y_pred=y_pred.round(),y_true=y_test)

print("Accuracy score is {}% ".format(accuracy_sc))
print("f1-score is {}% ".format(f1_sc))
print(classification_report(y_pred=y_pred.round(),y_true=y_test))

Accuracy score is 81.12% 
f1-score is 0.8062794992817567% 
              precision    recall  f1-score   support

           0       0.80      0.84      0.82      5000
           1       0.83      0.79      0.81      5000

    accuracy                           0.81     10000
   macro avg       0.81      0.81      0.81     10000
weighted avg       0.81      0.81      0.81     10000



## 5.Experiment

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_v = vect_layer(input_layer)
emb = keras.layers.Embedding(vocab_size, embedding_dim)(x_v)
x = LSTM(64, activation='relu', return_sequences=True)(emb)
x = GRU(64, activation='relu', return_sequences=True)(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(64, 'relu')(x)
x = keras.layers.Dense(32, 'relu')(x)
x = keras.layers.Dropout(0.2)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(optimizer=optimizer, loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_2 (TextV  (None, 30)               0         
 ectorization)                                                   
                                                                 
 embedding_8 (Embedding)     (None, 30, 128)           3840000   
                                                                 
 lstm_8 (LSTM)               (None, 30, 64)            49408     
                                                                 
 gru_8 (GRU)                 (None, 30, 64)            24960     
                                                                 
 flatten_8 (Flatten)         (None, 1920)              0         
                                                           

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

batch_size = 768
epochs = 2
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), callbacks=[es], epochs=epochs, batch_size=batch_size)

Epoch 1/2
1055/1055 [==============================] - 168s 157ms/step - loss: 0.4363 - accuracy: 0.7969 - val_loss: 0.4020 - val_accuracy: 0.8174
Epoch 2/2
1055/1055 [==============================] - 147s 140ms/step - loss: 0.3781 - accuracy: 0.8308 - val_loss: 0.3891 - val_accuracy: 0.8234


In [ ]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 3s 11ms/step - loss: 0.3857 - accuracy: 0.8257


[0.3857352137565613, 0.8256999850273132]

In [ ]:
y_pred=model.predict(x_test)

accuracy_sc = accuracy_score(y_pred=y_pred.round(),y_true=y_test)*100
f1_sc = f1_score(y_pred=y_pred.round(),y_true=y_test)

print("Accuracy score is {}% ".format(accuracy_sc))
print("f1-score is {}% ".format(f1_sc))
print(classification_report(y_pred=y_pred.round(),y_true=y_test))

Accuracy score is 82.57% 
f1-score is 0.8262386601535241% 
              precision    recall  f1-score   support

           0       0.83      0.82      0.83      5000
           1       0.82      0.83      0.83      5000

    accuracy                           0.83     10000
   macro avg       0.83      0.83      0.83     10000
weighted avg       0.83      0.83      0.83     10000



## 6.Experiment

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_v = vect_layer(input_layer)
emb = keras.layers.Embedding(vocab_size, embedding_dim)(x_v)
x = Bidirectional(LSTM(64, activation='relu', return_sequences=True))(emb)
x = Bidirectional(GRU(64, activation='relu', return_sequences=True))(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(64, 'relu')(x)
x = keras.layers.Dense(32, 'relu')(x)
x = keras.layers.Dropout(0.2)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(optimizer=optimizer, loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_2 (TextV  (None, 30)               0         
 ectorization)                                                   
                                                                 
 embedding_9 (Embedding)     (None, 30, 128)           3840000   
                                                                 
 bidirectional_10 (Bidirecti  (None, 30, 128)          98816     
 onal)                                                           
                                                                 
 bidirectional_11 (Bidirecti  (None, 30, 128)          74496     
 onal)                                                           
                                                           

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

batch_size = 768
epochs = 2
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), callbacks=[es], epochs=epochs, batch_size=batch_size)

Epoch 1/2
1055/1055 [==============================] - 274s 256ms/step - loss: 0.4329 - accuracy: 0.7982 - val_loss: 0.3972 - val_accuracy: 0.8181
Epoch 2/2
1055/1055 [==============================] - 268s 254ms/step - loss: 0.3741 - accuracy: 0.8331 - val_loss: 0.3889 - val_accuracy: 0.8242


In [ ]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 6s 19ms/step - loss: 0.3896 - accuracy: 0.8247


[0.38958531618118286, 0.8246999979019165]

In [ ]:
y_pred=model.predict(x_test)

accuracy_sc = accuracy_score(y_pred=y_pred.round(),y_true=y_test)*100
f1_sc = f1_score(y_pred=y_pred.round(),y_true=y_test)

print("Accuracy score is {}% ".format(accuracy_sc))
print("f1-score is {}% ".format(f1_sc))
print(classification_report(y_pred=y_pred.round(),y_true=y_test))

Accuracy score is 82.47% 
f1-score is 0.8213958227203261% 
              precision    recall  f1-score   support

           0       0.81      0.84      0.83      5000
           1       0.84      0.81      0.82      5000

    accuracy                           0.82     10000
   macro avg       0.83      0.82      0.82     10000
weighted avg       0.83      0.82      0.82     10000



# 7 - 8 Vectorizer parameters

In [ ]:
embedding_dim = 128 # Dimension of embedded representation 
vocab_size = 100000 # Number of unique tokens in vocabulary
sequence_length = 30 # Output dimension after vectorizing 

vect_layer = TextVectorization(max_tokens=vocab_size, output_mode='int', output_sequence_length=sequence_length)
vect_layer.adapt(x_train)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.10, random_state=69, stratify = y_train)

## 7.Experiment

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_v = vect_layer(input_layer)
emb = keras.layers.Embedding(vocab_size, embedding_dim)(x_v)
x = LSTM(64, activation='relu', return_sequences=True)(emb)
x = LSTM(64, activation='relu', return_sequences=True)(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(64, 'relu')(x)
x = keras.layers.Dense(32, 'relu')(x)
x = keras.layers.Dropout(0.2)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(optimizer=optimizer, loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

Model: "model_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_13 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_3 (TextV  (None, 30)               0         
 ectorization)                                                   
                                                                 
 embedding_12 (Embedding)    (None, 30, 128)           12800000  
                                                                 
 lstm_13 (LSTM)              (None, 30, 64)            49408     
                                                                 
 lstm_14 (LSTM)              (None, 30, 64)            33024     
                                                                 
 flatten_12 (Flatten)        (None, 1920)              0         
                                                          

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

batch_size = 768
epochs = 2
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), callbacks=[es], epochs=epochs, batch_size=batch_size)

Epoch 1/2
1055/1055 [==============================] - 145s 135ms/step - loss: 0.4357 - accuracy: 0.7977 - val_loss: 0.3946 - val_accuracy: 0.8218
Epoch 2/2
1055/1055 [==============================] - 138s 131ms/step - loss: 0.3581 - accuracy: 0.8415 - val_loss: 0.3934 - val_accuracy: 0.8232


In [ ]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 3s 10ms/step - loss: 0.3944 - accuracy: 0.8203


[0.3944157063961029, 0.8202999830245972]

In [ ]:
y_pred=model.predict(x_test)

accuracy_sc = accuracy_score(y_pred=y_pred.round(),y_true=y_test)*100
f1_sc = f1_score(y_pred=y_pred.round(),y_true=y_test)

print("Accuracy score is {}% ".format(accuracy_sc))
print("f1-score is {}% ".format(f1_sc))
print(classification_report(y_pred=y_pred.round(),y_true=y_test))

Accuracy score is 82.03% 
f1-score is 0.8176189992895565% 
              precision    recall  f1-score   support

           0       0.81      0.83      0.82      5000
           1       0.83      0.81      0.82      5000

    accuracy                           0.82     10000
   macro avg       0.82      0.82      0.82     10000
weighted avg       0.82      0.82      0.82     10000



## 8.Experiment

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_v = vect_layer(input_layer)
emb = keras.layers.Embedding(vocab_size, embedding_dim)(x_v)
x = Bidirectional(LSTM(128, activation='relu', return_sequences=True))(emb)
x = Bidirectional(LSTM(64, activation='relu', return_sequences=True))(x)
x = Bidirectional(GRU(64, activation='relu', return_sequences=True))(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(64, 'relu')(x)
x = keras.layers.Dense(32, 'relu')(x)
x = keras.layers.Dropout(0.2)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(optimizer=optimizer, loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

Model: "model_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_3 (TextV  (None, 30)               0         
 ectorization)                                                   
                                                                 
 embedding_11 (Embedding)    (None, 30, 128)           12800000  
                                                                 
 bidirectional_12 (Bidirecti  (None, 30, 256)          263168    
 onal)                                                           
                                                                 
 bidirectional_13 (Bidirecti  (None, 30, 128)          164352    
 onal)                                                           
                                                          

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

batch_size = 768
epochs = 2
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), callbacks=[es], epochs=epochs, batch_size=batch_size)

Epoch 1/2
1055/1055 [==============================] - 389s 364ms/step - loss: 0.4330 - accuracy: 0.7981 - val_loss: 0.3933 - val_accuracy: 0.8226
Epoch 2/2
1055/1055 [==============================] - 377s 357ms/step - loss: 0.3554 - accuracy: 0.8434 - val_loss: 0.3915 - val_accuracy: 0.8247


In [ ]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 9s 28ms/step - loss: 0.3934 - accuracy: 0.8258


[0.39340779185295105, 0.8258000016212463]

In [ ]:
y_pred=model.predict(x_test)

accuracy_sc = accuracy_score(y_pred=y_pred.round(),y_true=y_test)*100
f1_sc = f1_score(y_pred=y_pred.round(),y_true=y_test)

print("Accuracy score is {}% ".format(accuracy_sc))
print("f1-score is {}% ".format(f1_sc))
print(classification_report(y_pred=y_pred.round(),y_true=y_test))

Accuracy score is 82.58% 
f1-score is 0.8274905921964745% 
              precision    recall  f1-score   support

           0       0.83      0.82      0.82      5000
           1       0.82      0.84      0.83      5000

    accuracy                           0.83     10000
   macro avg       0.83      0.83      0.83     10000
weighted avg       0.83      0.83      0.83     10000



# Save model

In [ ]:
model.save('lstm_forth_done')

INFO:tensorflow:Assets written to: lstm_forth_done/assets


INFO:tensorflow:Assets written to: lstm_forth_done/assets


In [ ]:
!zip -r /content/LSTMforthD.zip /content/lstm_forth_done/

  adding: content/lstm_forth_done/ (stored 0%)
  adding: content/lstm_forth_done/saved_model.pb (deflated 78%)
  adding: content/lstm_forth_done/assets/ (stored 0%)
  adding: content/lstm_forth_done/variables/ (stored 0%)
  adding: content/lstm_forth_done/variables/variables.index (deflated 66%)
  adding: content/lstm_forth_done/variables/variables.data-00000-of-00001 (deflated 7%)
  adding: content/lstm_forth_done/keras_metadata.pb (deflated 90%)
